In [ ]:
!pip install -q efficientnet
!pip install tensorflow_addons
import re
import os
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import efficientnet.tfkeras as efn
from sklearn import metrics
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
from tqdm.notebook import tqdm

from kaggle_datasets import KaggleDatasets

In [ ]:
#!git clone https://github.com/hoangthang1607/nfnets-Tensorflow-2
#!touch ./nfnets-Tensorflow-2/__init__.py    
#!mv nfnets-Tensorflow-2 tfnfnet

In [ ]:
#!cp ../input/nfnet-base/tfnfnet ./tfnfnet -r

In [ ]:
!git clone https://github.com/Jannoshh/simple-sam
!mv simple-sam sam
import sam
import sam.sam
from sam.sam import sam_train_step

In [ ]:
import pandas as pd
from kaggle_datasets import KaggleDatasets
GCS_PATH = KaggleDatasets().get_gcs_path('shopee-product-matching')
df = pd.read_csv(GCS_PATH+'/train.csv',usecols=[4])
s=set()
res=df.values
for ele in res:
    s.add(ele[0])
print(len(s))


In [ ]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_PATH = KaggleDatasets().get_gcs_path('shopee-tfrecord-512x512')

# Configuration
EPOCHS = 20
BATCH_SIZE = 32 * strategy.num_replicas_in_sync
AUG_BATCH=BATCH_SIZE
IMAGE_SIZE = [512, 512]
# Seed
SEED = 42
# Learning rate
LR = 0.001
# Verbosity
VERBOSE = 2
# Number of classes
N_CLASSES = len(s)
# Number of folds
FOLDS = 5

# Training filenames directory
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/*.tfrec')
# Test filenames directory
TEST_FILENAMES = tf.io.gfile.glob(KaggleDatasets().get_gcs_path('shopee-product-matching')+'/test_images/*.jpg')

In [ ]:
MIXED_PRECISION = False
XLA_ACCELERATE = False
if MIXED_PRECISION:
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')
def resizemix(image,label,PROBABLITY = 1.0):
    DIM = IMAGE_SIZE[0]
    #CLASSES = 104
    classes=N_CLASSES
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],classes)
            lab2 = tf.one_hot(label[k],classes)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,classes))
    return image2,label2
def cutmix(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMAGE_SIZE[0]
    #CLASSES = 104
    classes=N_CLASSES
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],classes)
            lab2 = tf.one_hot(label[k],classes)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,classes))
    return image2,label2
def mixup(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with mixup applied
    DIM = IMAGE_SIZE[0]
    #CLASSES = 104
    classes=N_CLASSES
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO MIXUP WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        # MAKE CUTMIX LABEL
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j], classes)
            lab2 = tf.one_hot(label[k], classes)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH, classes))
    return image2,label2
def mixup_transform(posting_id, image, label, matches):
    # THIS FUNCTION APPLIES BOTH CUTMIX AND MIXUP
    DIM = IMAGE_SIZE[0]
    classes=N_CLASSES
    #CLASSES = 104
    SWITCH = 0.5
    CUTMIX_PROB = 0.666
    MIXUP_PROB = 0.666
    # FOR SWITCH PERCENT OF TIME WE DO CUTMIX AND (1-SWITCH) WE DO MIXUP
    image2, label2 = cutmix(image, label, CUTMIX_PROB)
    image3, label3 = mixup(image, label, MIXUP_PROB)
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
        imgs.append(P*image2[j,]+(1-P)*image3[j,])
        labs.append(P*label2[j,]+(1-P)*label3[j,])
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label4 = tf.reshape(tf.stack(labs),(AUG_BATCH,classes))
    
    return posting_id,image4,label4,matches

In [ ]:
# Function to get our f1 score

def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
def arcface_format(posting_id, image, label_group, matches):
    return posting_id, {'inp1': image, 'inp2': label_group}, label_group, matches
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))
def transform(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 60. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
    image=tf.reshape(d,[DIM,DIM,3])
    p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
            
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > 0.75:
        image = tf.image.transpose(image)
        
        
    # Pixel-level transforms
    if p_pixel_1 >= 0.4:
        image = tf.image.random_saturation(image, lower = 0.7, upper = 1.3)
    if p_pixel_2 >= 0.4:
        image = tf.image.random_contrast(image, lower = 0.8, upper = 1.2)
    if p_pixel_3 >= 0.4:
        image = tf.image.random_brightness(image, max_delta = 0.1)
    return image,label
# Data augmentation function
def data_augment(posting_id, image, label_group, matches):
    #image,label_group=transform(image,label_group)
    """image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_hue(image, 0.01)
    image = tf.image.random_saturation(image, 0.70, 1.30)
    image = tf.image.random_contrast(image, 0.80, 1.20)
    image = tf.image.random_brightness(image, 0.10)"""
    image,label_group=transform(image,label_group)
    return posting_id, image, label_group, matches

# Function to decode our images
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

# This function parse our images and also get the target variable
def read_labeled_tfrecord(example,onehot=False):
    LABELED_TFREC_FORMAT = {
        "posting_id": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "label_group": tf.io.FixedLenFeature([], tf.int64),
        "matches": tf.io.FixedLenFeature([], tf.string)
    }

    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    posting_id = example['posting_id']
    image = decode_image(example['image'])
    if onehot:
        label_group = tf.one_hot(tf.cast(example['label_group'], tf.int32), depth = N_CLASSES)
    else:
        label_group = tf.cast(example['label_group'], tf.int32)
    matches = example['matches']
    return posting_id, image, label_group, matches
from functools import partial
# This function loads TF Records and parse them into tensors
def load_dataset(filenames, ordered = False,_onehot=False):
    
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False 
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(partial(read_labeled_tfrecord,onehot=_onehot), num_parallel_calls = AUTO) 
    return dataset

# This function is to get our training tensors
def get_training_dataset(filenames, ordered = False,mixup=False):
    dataset = load_dataset(filenames, ordered = ordered,_onehot=mixup)
    dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    if mixup==False:
        dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    dataset = dataset.repeat()
    if mixup: 
        dataset = dataset.batch(BATCH_SIZE)
        dataset = dataset.map(mixup_transform, num_parallel_calls=AUTO)
        dataset.unbatch()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

# This function is to get our validation tensors
def get_validation_dataset(filenames, ordered = True,mixup=False):
    dataset = load_dataset(filenames, ordered = ordered,_onehot=mixup)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) 
    return dataset
def get_test_dataset(filenames, ordered = True):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) 
    return dataset
# Function to count how many photos we have in
def count_data_items(filenames):
    # The number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
print(f'Dataset: {NUM_TRAINING_IMAGES} training images')

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
# Function for a custom learning rate scheduler with warmup and decay
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.000005 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback

# Arcmarginproduct class keras layer
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output



In [ ]:
from bayes_opt import BayesianOptimization
def buildModel(m):
    model = get_model()
    train_and_evaluate()
def bayesOpt():
    pbounds = {
        'mm':(0.1,0.9),
        'ss':(5,20)
    }
    optimizer = BayesianOptimization(f=getModel, pbounds=pbounds)
    optimizer.maximize(init_points=5, n_iter=10, acq='ucb')
    return optimizer

In [ ]:
!git clone https://github.com/Jannoshh/simple-sam
#!touch ./nfnets-Tensorflow-2/__init__.py    
#!mv nfnets-Tensorflow-2 tfnfnet

In [ ]:
import math
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras.initializers import Constant
from tensorflow.python.keras.utils import tf_utils


def _resolve_training(layer, training):
    if training is None:
        training = K.learning_phase()
    if isinstance(training, int):
        training = bool(training)
    if not layer.trainable:
        # When the layer is not trainable, override the value
        training = False
    return training#tf_utils.constant_value(training)

In [ ]:
import tensorflow.keras.layers
class AdaCos(Layer):
    """
    Implementation of AdaCos layer. Reference: https://arxiv.org/abs/1905.00292
    
    Arguments:
      num_classes: number of classes to classify
      is_dynamic: if False, use Fixed AdaCos. Else, use Dynamic Adacos.
      regularizer: weights regularizer
    """
    def __init__(self,
                 num_classes,
                 is_dynamic=True,
                 regularizer=None,
                 **kwargs):

        super().__init__(**kwargs)
        self._n_classes = num_classes
        self._init_s = math.sqrt(2) * math.log(num_classes - 1)
        self._is_dynamic = is_dynamic
        self._regularizer = regularizer

    def build(self, input_shape):
        embedding_shape, label_shape = input_shape
        self._w = self.add_weight(name='W',shape=(embedding_shape[-1], self._n_classes),
                                  initializer='glorot_uniform',
                                  trainable=True,
                                  regularizer=self._regularizer)
        if self._is_dynamic:
            self._s = self.add_weight(name='S',shape=(),
                                      initializer=Constant(self._init_s),
                                      trainable=False,
                                      aggregation=tf.VariableAggregation.MEAN)

    def call(self, inputs, training=None):
        embedding, label = inputs
        label = tf.cast(label, dtype=tf.int32)

        # Squeezing is necessary for Keras. It expands the dimension to (n, 1)
        label = tf.reshape(label, [-1])

        # Normalize features and weights and compute dot product
        x = tf.nn.l2_normalize(embedding, axis=1)
        w = tf.nn.l2_normalize(self._w, axis=0)
        logits = tf.matmul(x, w)

        # Fixed AdaCos
        is_dynamic = self._is_dynamic#tf_utils.constant_value(self._is_dynamic)
        if not is_dynamic:
            # _s is not created since we are not in dynamic mode
            output = tf.multiply(self._init_s, logits)
            return output

        training = _resolve_training(self, training)
        if not training:
            # We don't have labels to update _s if we're not in training mode
            return self._s * logits
        else:
            theta = tf.math.acos(
                    K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
            one_hot = tf.one_hot(label, depth=self._n_classes)
            b_avg = tf.where(one_hot < 1.0,
                             tf.exp(self._s * logits),
                             tf.zeros_like(logits))
            b_avg = tf.reduce_mean(tf.reduce_sum(b_avg, axis=1))
            theta_class = tf.gather_nd(
                    theta,
                    tf.stack([
                        tf.range(tf.shape(label)[0]),
                        tf.cast(label, tf.int32)
                    ], axis=1))
            mid_index = tf.shape(theta_class)[0] // 2 + 1
            theta_med = tf.nn.top_k(theta_class, mid_index).values[-1]

            # Since _s is not trainable, this assignment is safe. Also,
            # tf.function ensures that this will run in the right order.
            self._s.assign(
                    tf.math.log(b_avg) /
                    tf.math.cos(tf.minimum(math.pi/4, theta_med)))

            # Return scaled logits
            return self._s * logits

In [ ]:
class ArcFace(Layer):
    """
    Implementation of ArcFace layer. Reference: https://arxiv.org/abs/1801.07698
    
    Arguments:
      num_classes: number of classes to classify
      s: scale factor
      m: margin
      regularizer: weights regularizer
    """
    def __init__(self,
                 num_classes,
                 s=30.0,
                 m=0.5,
                 regularizer=None,
                 name='arcface',
                 **kwargs):
        
        super().__init__(name=name, **kwargs)
        self._n_classes = num_classes
        self._s = float(s)
        self._m = float(m)
        self._regularizer = regularizer

    def build(self, input_shape):
        embedding_shape, label_shape = input_shape
        self._w = self.add_weight(shape=(embedding_shape[-1], self._n_classes),
                                  initializer='glorot_uniform',
                                  trainable=True,
                                  regularizer=self._regularizer,
                                  name='cosine_weights')

    def call(self, inputs, training=None):
        """
        During training, requires 2 inputs: embedding (after backbone+pool+dense),
        and ground truth labels. The labels should be sparse (and use
        sparse_categorical_crossentropy as loss).
        """
        embedding, label = inputs

        # Squeezing is necessary for Keras. It expands the dimension to (n, 1)
        label = tf.reshape(label, [-1], name='label_shape_correction')

        # Normalize features and weights and compute dot product
        x = tf.nn.l2_normalize(embedding, axis=1, name='normalize_prelogits')
        w = tf.nn.l2_normalize(self._w, axis=0, name='normalize_weights')
        cosine_sim = tf.matmul(x, w, name='cosine_similarity')

        training = resolve_training_flag(self, training)
        if not training:
            # We don't have labels if we're not in training mode
            return self._s * cosine_sim
        else:
            one_hot_labels = tf.one_hot(label,
                                        depth=self._n_classes,
                                        name='one_hot_labels')
            theta = tf.math.acos(K.clip(
                    cosine_sim, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
            selected_labels = tf.where(tf.greater(theta, math.pi - self._m),
                                       tf.zeros_like(one_hot_labels),
                                       one_hot_labels,
                                       name='selected_labels')
            final_theta = tf.where(tf.cast(selected_labels, dtype=tf.bool),
                                   theta + self._m,
                                   theta,
                                   name='final_theta')
            output = tf.math.cos(final_theta, name='cosine_sim_with_margin')
            return self._s * output

In [ ]:
!pip uninstall -y nfnets-keras
!pip install -U git+https://github.com/yamacrypt/nfnets-keras
import nfnets_keras
import importlib
importlib.reload(nfnets_keras)

In [ ]:
import importlib
importlib.reload(nfnets_keras)

In [ ]:
#!pip install git+https://github.com/prateekkrjain/nfnets-keras

In [ ]:
from efficientnet.keras import EfficientNetB5,EfficientNetB4
import h5py
#from nfnets_keras import NFNetF0
#from  tfnfnet.nfnet import NFNet
from keras.utils.conv_utils import convert_kernel
# Function to create our EfficientNetB3 model
def premodel():
        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        x = EfficientNetB4(weights=None, include_top = False)(inp)
        #x=tf.keras.applications.DenseNet201(include_top = False)(inp)
        #x = NFNet(
        #num_classes=1000,
        #variant="F0",
        #label_smoothing=0,
        #)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x= tf.keras.layers.BatchNormalization()(x)
        #x=tf.keras.applications.ResNet50(weights='imagenet',include_top=False,classes=N_CLASSES)(inp);
        x = tf.keras.layers.Dense(N_CLASSES)(x)
        #x=cos_loss(x,label,N_CLASSES)
        #x = margin([x, label])
        #output=ArcFaceLayer0(N_CLASSES)(x)
        output = tf.keras.layers.Softmax(dtype='float32')(x)
        embeddingmodel = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
        #x=tf.keras.applications.ResNet50(weights='imagenet',include_top=False,classes=N_CLASSES)(inp);
        #x = tf.keras.layers.Dense(N_CLASSES)(x)
        #x=cos_loss(x,label,N_CLASSES)
        #output=ArcFaceLayer0(N_CLASSES)(x)
        weights_path = '../input/effb4base/effnetB4-sam.h5'
        #before=get_model2()
        embeddingmodel.load_weights(weights_path)
        return embeddingmodel
#pre_model=premodel()

class SAMModel(tf.keras.models.Model):
    def train_step(self, data):
        return sam_train_step(self, data)
import gc
def get_model(ss=10,mm=0.5):
    with strategy.scope():
        adacos=AdaCos(N_CLASSES, name='AdaCos', )
        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        #x=tf.keras.applications.DenseNet201(include_top = False)(inp)
        x = EfficientNetB4(weights='noisy-student', include_top = False)(inp)
        #x = EfficientNetB5(weights=None, include_top = False)(inp)
        #x = NFNet(
        ##num_classes=1000,
        #variant="F0",
        #label_smoothing=0,
        #)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x= tf.keras.layers.BatchNormalization()(x)
        #x=tf.keras.applications.ResNet50(weights='imagenet',include_top=False,classes=N_CLASSES)(inp);
        #x = tf.keras.layers.Dense(N_CLASSES)(x)
        #x=cos_loss(x,label,N_CLASSES)
        x = adacos((x,label),True)#margin([x, label])
        #output=ArcFaceLayer0(N_CLASSES)(x)
        output = tf.keras.layers.Softmax(dtype='float32')(x)
        model = SAMModel(inputs = [inp, label], outputs = [output])
        pre=premodel()
        model.layers[1].set_weights(pre.layers[1].get_weights())
        print('Model loaded.')
        del pre
        gc.collect()

        opt = tf.keras.optimizers.Adam(learning_rate = LR)
        radam = tfa.optimizers.RectifiedAdam()
        sgd=tf.keras.optimizers.SGD(learning_rate=LR)
        #tf.keras.losses.CosineSimilarity() tf.keras.losses.SparseCategoricalCrossentropy()
        model.compile(
            optimizer = radam,
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
            ) 
        
        return model
def get_model2():
    with strategy.scope():
        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3),name='inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        x =EfficientNetB4(weights='noisy-student', include_top = False)(inp)
        #x=tf.keras.applications.DenseNet201(include_top = False)(inp)
        #x = nfnets_keras.NFNetF0(num_classes=1000,include_top=False)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x= tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(N_CLASSES)(x)
        #x=cos_loss(x,label,N_CLASSES)
        #x = margin([x, label])
        #output=ArcFaceLayer0(N_CLASSES)(x)
        output = tf.keras.layers.Softmax(dtype='float32')(x)
        model = SAMModel(inputs = [inp,label], outputs = [output])
        opt = tf.keras.optimizers.Adam(learning_rate = LR)
        radam = tfa.optimizers.RectifiedAdam()
        sgd=tf.keras.optimizers.SGD()
        #tf.keras.losses.CosineSimilarity() tf.keras.losses.SparseCategoricalCrossentropy()
        model.compile(
            optimizer = radam,
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
            ) 
        
        return model

def train_and_evaluate(save_file=f'EfficientNetB5_adacos_512_42.h5',mixup=False):
    # Seed everything
    seed_everything(SEED)
    print('\n')
    print('-'*50)
    train, valid = train_test_split(TRAINING_FILENAMES, shuffle = True, random_state = SEED)
    train_dataset = get_training_dataset(train, ordered = False,mixup=mixup)
    val_dataset = get_validation_dataset(valid, ordered = True,mixup=mixup)
    STEPS_PER_EPOCH = count_data_items(train) // BATCH_SIZE
    del train
    del valid
    if(mixup==False):
        train_dataset = train_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
        val_dataset = val_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
    else:
        train_dataset = train_dataset.map(lambda posting_id, image, label_group, matches: image)
        val_dataset = val_dataset.map(lambda posting_id, image, label_group, matches: image)
        
    #K.clear_session()
    #model=get_model()
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.00001,verbose=1)
    # Model checkpoint
    early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                              patience = 3, mode = 'min', verbose = 1,
                           restore_best_weights = True)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_file, 
                                                    monitor = 'val_loss', 
                                                    save_best_only = True,
                                                    save_weights_only = True, 
                                                    mode = 'min')
    #, get_lr_callback()
    history = model.fit(train_dataset,
                        steps_per_epoch = STEPS_PER_EPOCH,
                        epochs = EPOCHS,
                        callbacks = [early_stop,reduce_lr,checkpoint], 
                        validation_data = val_dataset,
                        )
    del train_dataset
    del val_dataset
    return model

In [ ]:
#m=SAMModel()
#print(pre_model.layers)
BATCH_SIZE = 32 * strategy.num_replicas_in_sync

In [ ]:
#K.clear_session()
#model = get_model2()
#model.summary()
#train_and_evaluate(f'effnetB4-sam.h5')

In [ ]:
import gc
K.clear_session()
gc.collect()
model = get_model(ss=64,mm=0.7)
#model.layers[1].tranable=False
train_and_evaluate()

In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold
import math, re, os
import gc
import tensorflow as tf
#from tensorflow.keras.layers import *
import numpy as np
from tensorflow import keras
from functools import partial

from tensorflow.keras.applications.imagenet_utils import decode_predictions
import tensorflow_addons as tfa
FOLD=4
kf =  KFold(n_splits=FOLD, shuffle=True, random_state = SEED)
#GCS_PATH = '/kaggle/input/cassava-leaf-disease-tfrecords-center-512x512'
i=0
#for n_fold, (train, test) in enumerate(cv.split(X)):

for train_index, test_index in kf.split(TRAINING_FILENAMES):
    K.clear_session()
    gc.collect()
    model = get_model(ss=20,mm=0.7)
    seed_everything(SEED)
        
    train, valid = [TRAINING_FILENAMES[i] for i in train_index],[TRAINING_FILENAMES[i] for i in test_index]#train_test_split(TRAINING_FILENAMES, shuffle = True, random_state = SEED)
    print(f'fold={i}')
    train_dataset = get_training_dataset(train, ordered = False)
    train_dataset = train_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
    val_dataset = get_validation_dataset(valid, ordered = True)
    val_dataset = val_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
    STEPS_PER_EPOCH = count_data_items(train) // BATCH_SIZE
    del train
    del valid
    #K.clear_session()
    #model=get_model()
    # Model checkpoint
    save_file=f'eff_adacos_model_fold{i}.h5'
    i+=1
    early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                              patience = 3, mode = 'min', verbose = 1,
                           restore_best_weights = True)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_file, 
                                                    monitor = 'val_loss', 
                                                    save_best_only = True,
                                                    save_weights_only = True, 
                                                    mode = 'min')
    #, get_lr_callback()
    history = model.fit(train_dataset,
                        steps_per_epoch = STEPS_PER_EPOCH,
                        epochs = EPOCHS,
                        callbacks = [checkpoint,early_stop,get_lr_callback()], 
                        validation_data = val_dataset,
                        )
    del train_dataset
    del val_dataset


In [ ]:
class BiTemperedLogisticLoss(tf.keras.losses.Loss):
    def __init__(self, t1, t2, lbl_smth, n_iter,weights):
      super(BiTemperedLogisticLoss, self).__init__()
      self.t1 = t1
      self.t2 = t2
      self.weights=weights
      self.lbl_smth = lbl_smth
      self.n_iter = n_iter

    def call(self, y_true, y_pred):
        nb_cl = len(self.weights)
        final_mask = K.zeros_like(y_pred[:, 0])
        y_pred_max = K.max(y_pred, axis=1)
        y_pred_max = K.reshape(y_pred_max, (K.shape(y_pred)[0], 1))
        y_pred_max_mat = K.equal(y_pred, y_pred_max)
        for c_p, c_t in product(range(nb_cl), range(nb_cl)):
            final_mask += (self.weights[c_t, c_p] * y_pred_max_mat[:, c_p] * y_true[:, c_t])
        return bi_tempered_logistic_loss(y_pred, y_true, self.t1, self.t2, self.lbl_smth, self.n_iter) * final_mask
def w_bi_tempered_logistic_loss(y_true, y_pred,weights,t1,t2,label_smoothing=0,n_iter=5 ):
        if(weights==None):
            return bi_tempered_logistic_loss(y_pred, y_true,t1,t2,label_smoothing,n_iter)
        nb_cl = len(weights)
        final_mask = K.zeros_like(y_pred[:, 0])
        y_pred_max = K.max(y_pred, axis=1)
        y_pred_max = K.reshape(y_pred_max, (K.shape(y_pred)[0], 1))
        y_pred_max_mat = K.equal(y_pred, y_pred_max)
        for c_p, c_t in product(range(nb_cl), range(nb_cl)):
            final_mask += (weights[c_t, c_p] * y_pred_max_mat[:, c_p] * y_true[:, c_t])
        return bi_tempered_logistic_loss(y_pred, y_true,t1,t2,label_smoothing,n_iter) * final_mask
class WeightBiTemperedLogisticLoss( keras.losses.CategoricalCrossentropy):
        def __init__(self,
                   from_logits=False,
                   label_smoothing=0,
                   reduction=keras.losses.Reduction.AUTO,
                   name='w_bi_tempered_logistic_loss',
                   t1=1.0,
                   t2=1.0,
                   weights=None):
            @dispatch.add_dispatch_support
            def loss_fun():
              return  partial(w_bi_tempered_logistic_loss, t1=t1,t2=t2,weights= weights)
            super().__init__(
            loss_fun,
            name=name,
            label_smoothing=label_smoothing
            )

In [ ]:
import os 
os.chdir('/kaggle/working')
from IPython.display import FileLink 
FileLink('./effnetB4-sam.h5')

In [ ]:
import os 
os.chdir('/kaggle/working')
from IPython.display import FileLink 
FileLink('./eff_adacos_model_fold3.h5')